In [1]:
random_seed=7
from numpy.random import seed
seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)

In [68]:
import tensorflow as tp
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
import pandas as pd
from keras import backend as K
# from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

In [3]:
from sklearn.model_selection import KFold # import KFold

In [4]:
#checking if GPU is used 
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [5]:
dataset = pd.read_csv("../../data/1.csv", delimiter=",")
#dataset

In [6]:
def mean_imputation(dataset): 
    """"""
    imputed_dataset=dataset.fillna(dataset.mean())
    return imputed_dataset

In [7]:
dataset=mean_imputation(dataset)
#dataset

In [8]:
#to _numpy() works for 0.24.2 + version of pandas
pd.__version__

'0.24.2'

In [9]:
#dataset_np_array=dataset.to_numpy()
dataset_np_array=dataset.values
dataset_np_array

array([[ 18. ,   8. , 307. , ...,  12. ,  70. ,   1. ],
       [ 15. ,   8. , 350. , ...,  11.5,  70. ,   1. ],
       [ 18. ,   8. , 318. , ...,  11. ,  70. ,   1. ],
       ...,
       [ 32. ,   4. , 135. , ...,  11.6,  82. ,   1. ],
       [ 28. ,   4. , 120. , ...,  18.6,  82. ,   1. ],
       [ 31. ,   4. , 119. , ...,  19.4,  82. ,   1. ]])

In [10]:
#split into input (X) and output (Y)
def splitXY(dataset): 
    """
    Takes numpy array as input and converts first column into Y and rest into X
    """
    Y=dataset[:,0]
    X=dataset[:,1:]
    return X,Y

X,Y= splitXY(dataset_np_array)

In [11]:
def r_squared(y_true, y_pred):
    
    sse=tp.reduce_sum(tp.square(tp.subtract(y_pred,y_true)))
    #using sst as y_true-y_mean 's squared mean
    y_mean= tp.reduce_mean(y_true)
    sst=tp.reduce_sum(tp.square(tp.subtract(y_true,y_mean)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    
    """
    m=tp.to_float(tp.size(y_true))
    y_true_sum_sq=(tp.square(tp.reduce_sum(y_true)))
    y_sq_mean=tp.divide(y_true_sum_sq,m)
    sst=tp.subtract(tp.tensordot(y_true,y_true,0),m*tp.square(tp.reduce_mean(y_true)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    """
    return r_square

In [12]:
def adj_r_squared(y_true, y_pred):
    sse=tp.reduce_sum(tp.square(tp.subtract(y_pred,y_true)))
    #using sst as y_true-y_mean 's squared mean
    y_mean= tp.reduce_mean(y_true)
    sst=tp.reduce_sum(tp.square(tp.subtract(y_true,y_mean)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    
    #n=model.count_params()#this part was working before but now is not..how to get model here though?
    #n=sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])
    n=tp.to_float(count)
    #m=tp.to_float(tp.size(y_true))
    rdf=tp.divide(tp.subtract(tp.to_float(instances),float(1)),tp.subtract(tp.to_float(instances),n))
    r_adj_square=tp.subtract(float(1),tp.tensordot(rdf,tp.subtract(float(1),r_square),0))
    return r_adj_square

In [36]:
def perceptronModel(X_dim=7,activation_name='sigmoid'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(1, input_dim=X_dim,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    global count
    count=model.count_params()
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    sgd=optimizers.SGD(lr=0.05, momentum=0.05, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [69]:
def neuralNetwork3LModel(X_dim,activation_name='relu'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(X_dim, input_dim=X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    global count
    count=model.count_params()#-X_dim+1
    sgd=optimizers.SGD(lr=0.05, momentum=0.05, decay=0.0, nesterov=False)
    rmsprop=keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
    model.compile(loss='mean_squared_error',optimizer=rmsprop,metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [70]:
def neuralNetwork5LModel(X_dim,activation_name='sigmoid'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(X_dim, input_dim=X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    global count
    count=model.count_params() #-3*X_dim +1
    print('Count of parameters :'+str(count))
    sgd=optimizers.SGD(lr=0.05, momentum=0.05, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [71]:
def fit_model(X,Y):
    
    (X_size,X_dim)=X.shape
    global instances
    instances=X_size
    #model =perceptronModel(X_dim)
    model =neuralNetwork3LModel(X_dim)
    #model =neuralNetwork5LModel(X_dim)#getting gpu sync failed .. internal error
    count=model.count_params() #-3*X_dim +1
    print('Count of parameters :'+str(count))
    model.summary()

    # Fit the model
    model.fit(X, Y, epochs=1000, batch_size=30)

    # evaluate the model
    # evaluate model with standardized dataset
    #estimator = KerasRegressor(build_fn=perceptronModel, epochs=100, batch_size=5, verbose=0)

    # kfold = KFold(n_splits=10, random_state=seed)
    # results = cross_val_score(estimator, X, Y, cv=kfold)
    # print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
    return model
model=fit_model(X,Y)

Count of parameters :64
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 56        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8         
Total params: 64
Trainable params: 64
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
406/406 [==============================] - 0s 604us/sample - loss: 6694393.9353 - mean_squared_error: 6694394.5000 - r_squared: -113170.5078 - adj_r_squared: -134017.9062
Epoch 2/1000
406/406 [==============================] - 0s 108us/sample - loss: 510073.8972 - mean_squared_error: 510073.9375 - r_squared: -9206.8350 - adj_r_squared: -10903.0146
Epoch 3/1000
406/406 [==============================] - 0s 123us/sample - loss: 29402.4839 - mean_squared_error: 29402.4824 - r_squared: -528.0504 - adj_r_

406/406 [==============================] - 0s 99us/sample - loss: 8408.7202 - mean_squared_error: 8408.7207 - r_squared: -147.1439 - adj_r_squared: -174.4335
Epoch 45/1000
406/406 [==============================] - 0s 113us/sample - loss: 7818.9880 - mean_squared_error: 7818.9878 - r_squared: -130.2125 - adj_r_squared: -154.3832
Epoch 46/1000
406/406 [==============================] - 0s 99us/sample - loss: 2700.0183 - mean_squared_error: 2700.0188 - r_squared: -54.0099 - adj_r_squared: -64.1433
Epoch 47/1000
406/406 [==============================] - 0s 104us/sample - loss: 7316.5691 - mean_squared_error: 7316.5698 - r_squared: -128.2419 - adj_r_squared: -152.0496
Epoch 48/1000
406/406 [==============================] - 0s 108us/sample - loss: 7007.0403 - mean_squared_error: 7007.0410 - r_squared: -124.1354 - adj_r_squared: -147.1866
Epoch 49/1000
406/406 [==============================] - 0s 94us/sample - loss: 5879.1768 - mean_squared_error: 5879.1768 - r_squared: -105.0019 - adj_r_

406/406 [==============================] - 0s 108us/sample - loss: 4273.4330 - mean_squared_error: 4273.4331 - r_squared: -78.0536 - adj_r_squared: -92.6161
Epoch 92/1000
406/406 [==============================] - 0s 108us/sample - loss: 5713.2164 - mean_squared_error: 5713.2163 - r_squared: -108.2602 - adj_r_squared: -128.3871
Epoch 93/1000
406/406 [==============================] - 0s 102us/sample - loss: 4709.9039 - mean_squared_error: 4709.9038 - r_squared: -75.8852 - adj_r_squared: -90.0483
Epoch 94/1000
406/406 [==============================] - 0s 99us/sample - loss: 4263.0871 - mean_squared_error: 4263.0869 - r_squared: -76.9452 - adj_r_squared: -91.3035
Epoch 95/1000
406/406 [==============================] - 0s 105us/sample - loss: 4520.9318 - mean_squared_error: 4520.9321 - r_squared: -79.8845 - adj_r_squared: -94.7843
Epoch 96/1000
406/406 [==============================] - 0s 102us/sample - loss: 4224.8553 - mean_squared_error: 4224.8555 - r_squared: -84.3246 - adj_r_squar

406/406 [==============================] - 0s 108us/sample - loss: 3258.4896 - mean_squared_error: 3258.4893 - r_squared: -63.4588 - adj_r_squared: -75.3328
Epoch 139/1000
406/406 [==============================] - 0s 100us/sample - loss: 3677.8350 - mean_squared_error: 3677.8352 - r_squared: -64.3238 - adj_r_squared: -76.3572
Epoch 140/1000
406/406 [==============================] - 0s 102us/sample - loss: 3147.0957 - mean_squared_error: 3147.0959 - r_squared: -56.1279 - adj_r_squared: -66.6515
Epoch 141/1000
406/406 [==============================] - 0s 123us/sample - loss: 3114.8226 - mean_squared_error: 3114.8228 - r_squared: -52.8442 - adj_r_squared: -62.7629
Epoch 142/1000
406/406 [==============================] - 0s 108us/sample - loss: 3169.0399 - mean_squared_error: 3169.0400 - r_squared: -54.3843 - adj_r_squared: -64.5866
Epoch 143/1000
406/406 [==============================] - 0s 111us/sample - loss: 2099.1009 - mean_squared_error: 2099.1011 - r_squared: -38.8991 - adj_r_s

406/406 [==============================] - 0s 107us/sample - loss: 1519.9523 - mean_squared_error: 1519.9523 - r_squared: -22.7789 - adj_r_squared: -27.1593
Epoch 186/1000
406/406 [==============================] - 0s 103us/sample - loss: 1611.6179 - mean_squared_error: 1611.6179 - r_squared: -30.6807 - adj_r_squared: -36.5166
Epoch 187/1000
406/406 [==============================] - 0s 102us/sample - loss: 1527.4407 - mean_squared_error: 1527.4409 - r_squared: -25.7942 - adj_r_squared: -30.7299
Epoch 188/1000
406/406 [==============================] - 0s 99us/sample - loss: 1311.9701 - mean_squared_error: 1311.9700 - r_squared: -25.1049 - adj_r_squared: -29.9137
Epoch 189/1000
406/406 [==============================] - 0s 124us/sample - loss: 1897.7472 - mean_squared_error: 1897.7472 - r_squared: -32.9857 - adj_r_squared: -39.2463
Epoch 190/1000
406/406 [==============================] - 0s 101us/sample - loss: 1369.1439 - mean_squared_error: 1369.1439 - r_squared: -28.5522 - adj_r_sq

...

406/406 [==============================] - 0s 118us/sample - loss: 19.4869 - mean_squared_error: 19.4869 - r_squared: 0.6488 - adj_r_squared: 0.5841
Epoch 778/1000
406/406 [==============================] - 0s 104us/sample - loss: 20.0078 - mean_squared_error: 20.0078 - r_squared: 0.6351 - adj_r_squared: 0.5679
Epoch 779/1000
406/406 [==============================] - 0s 99us/sample - loss: 24.1608 - mean_squared_error: 24.1608 - r_squared: 0.5292 - adj_r_squared: 0.4424
Epoch 780/1000
406/406 [==============================] - 0s 102us/sample - loss: 17.8751 - mean_squared_error: 17.8751 - r_squared: 0.6614 - adj_r_squared: 0.5990
Epoch 781/1000
406/406 [==============================] - 0s 115us/sample - loss: 21.0905 - mean_squared_error: 21.0905 - r_squared: 0.6174 - adj_r_squared: 0.5470
Epoch 782/1000
406/406 [==============================] - 0s 102us/sample - loss: 19.9067 - mean_squared_error: 19.9067 - r_squared: 0.6524 - adj_r_squared: 0.5883
Epoch 783/1000
406/406 [========

406/406 [==============================] - 0s 100us/sample - loss: 19.3655 - mean_squared_error: 19.3655 - r_squared: 0.5880 - adj_r_squared: 0.5121
Epoch 828/1000
406/406 [==============================] - 0s 108us/sample - loss: 20.6322 - mean_squared_error: 20.6322 - r_squared: 0.6297 - adj_r_squared: 0.5615
Epoch 829/1000
406/406 [==============================] - 0s 104us/sample - loss: 20.7889 - mean_squared_error: 20.7889 - r_squared: 0.6182 - adj_r_squared: 0.5478
Epoch 830/1000
406/406 [==============================] - 0s 99us/sample - loss: 20.0089 - mean_squared_error: 20.0089 - r_squared: 0.6395 - adj_r_squared: 0.5730
Epoch 831/1000
406/406 [==============================] - 0s 89us/sample - loss: 21.3244 - mean_squared_error: 21.3244 - r_squared: 0.6137 - adj_r_squared: 0.5425
Epoch 832/1000
406/406 [==============================] - 0s 99us/sample - loss: 18.5742 - mean_squared_error: 18.5742 - r_squared: 0.6460 - adj_r_squared: 0.5807
Epoch 833/1000
406/406 [==========

406/406 [==============================] - 0s 89us/sample - loss: 19.7187 - mean_squared_error: 19.7187 - r_squared: 0.6636 - adj_r_squared: 0.6017
Epoch 878/1000
406/406 [==============================] - 0s 108us/sample - loss: 16.6408 - mean_squared_error: 16.6408 - r_squared: 0.7061 - adj_r_squared: 0.6520
Epoch 879/1000
406/406 [==============================] - 0s 108us/sample - loss: 21.1950 - mean_squared_error: 21.1950 - r_squared: 0.6270 - adj_r_squared: 0.5583
Epoch 880/1000
406/406 [==============================] - 0s 99us/sample - loss: 20.9343 - mean_squared_error: 20.9343 - r_squared: 0.6311 - adj_r_squared: 0.5631
Epoch 881/1000
406/406 [==============================] - 0s 108us/sample - loss: 21.1757 - mean_squared_error: 21.1757 - r_squared: 0.6104 - adj_r_squared: 0.5387
Epoch 882/1000
406/406 [==============================] - 0s 102us/sample - loss: 18.7462 - mean_squared_error: 18.7462 - r_squared: 0.6241 - adj_r_squared: 0.5548
Epoch 883/1000
406/406 [=========

406/406 [==============================] - 0s 103us/sample - loss: 18.9095 - mean_squared_error: 18.9095 - r_squared: 0.6570 - adj_r_squared: 0.5938
Epoch 928/1000
406/406 [==============================] - 0s 108us/sample - loss: 17.3276 - mean_squared_error: 17.3276 - r_squared: 0.6902 - adj_r_squared: 0.6332
Epoch 929/1000
406/406 [==============================] - 0s 104us/sample - loss: 21.4591 - mean_squared_error: 21.4591 - r_squared: 0.6305 - adj_r_squared: 0.5624
Epoch 930/1000
406/406 [==============================] - 0s 99us/sample - loss: 19.5815 - mean_squared_error: 19.5815 - r_squared: 0.6537 - adj_r_squared: 0.5899
Epoch 931/1000
406/406 [==============================] - 0s 100us/sample - loss: 19.3703 - mean_squared_error: 19.3703 - r_squared: 0.6568 - adj_r_squared: 0.5935
Epoch 932/1000
406/406 [==============================] - 0s 97us/sample - loss: 19.7645 - mean_squared_error: 19.7645 - r_squared: 0.6432 - adj_r_squared: 0.5774
Epoch 933/1000
406/406 [=========

406/406 [==============================] - 0s 128us/sample - loss: 17.9230 - mean_squared_error: 17.9230 - r_squared: 0.6872 - adj_r_squared: 0.6296
Epoch 978/1000
406/406 [==============================] - 0s 108us/sample - loss: 22.2259 - mean_squared_error: 22.2259 - r_squared: 0.5975 - adj_r_squared: 0.5234
Epoch 979/1000
406/406 [==============================] - 0s 108us/sample - loss: 17.8608 - mean_squared_error: 17.8608 - r_squared: 0.6866 - adj_r_squared: 0.6288
Epoch 980/1000
406/406 [==============================] - 0s 111us/sample - loss: 20.2638 - mean_squared_error: 20.2638 - r_squared: 0.6396 - adj_r_squared: 0.5732
Epoch 981/1000
406/406 [==============================] - 0s 115us/sample - loss: 19.0509 - mean_squared_error: 19.0509 - r_squared: 0.6779 - adj_r_squared: 0.6186
Epoch 982/1000
406/406 [==============================] - 0s 111us/sample - loss: 20.0310 - mean_squared_error: 20.0310 - r_squared: 0.6349 - adj_r_squared: 0.5677
Epoch 983/1000
406/406 [=======

In [72]:
def eval_model(X,Y,model):
#model evaluation.

    scores = model.evaluate(X, Y, verbose=0)
        
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    return scores
eval_model(X,Y,model)

mean_squared_error: 16.64%
r_squared: 42.16%
adj_r_squared: 31.51%


[16.63968109732191, 16.639679, 0.42163068, 0.31508893]

In [50]:
#showing error... this method is for classification..
#do test train___cross val is going bad..
def cross_validation():
    ##try cross validate for regressor, but without one..
    # define 10-fold cross validation test harness
    kf = KFold(n_splits=2, shuffle=False, random_state=seed)
    cvscores = []
    for train_index, test_index in kf.split(X):
        model=fit_model(X[train_index],Y[train_index])
        # evaluate the model
        scores = eval_model(X[test_index],Y[test_index],model)
        dict_scores={}
        dict_scores[model.metrics_names[1]]=scores[1]
        dict_scores[model.metrics_names[2]]=scores[2]
        dict_scores[model.metrics_names[3]]=scores[3]
        cvscores.append(dict_scores)
    print("SCORES:-")
    for dict_scores in cvscores:
        for k,v in dict_scores.items():
            print(str(k)+":"+str(v))
cross_validation()

Count of parameters :64
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 7)                 56        
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 8         
Total params: 64
Trainable params: 64
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
203/203 [==============================] - 1s 4ms/step - loss: 216305197916962593190719782912.0000 - mean_squared_error: 216305197916962593190719782912.0000 - r_squared: -3289038545349192455282491392.0000 - adj_r_squared: -4779753867412914443789533184.0000
Epoch 2/1000
203/203 [==============================] - 0s 138us/step - loss: 1989021845406938879850708992.0000 - mean_squared_error: 1989021845406938879850708992.0000 - r_squared: -33342228814845278840422400.0000 - adj_r_squared: -4845417451735619

203/203 [==============================] - 0s 118us/step - loss: 59.1448 - mean_squared_error: 59.1448 - r_squared: -0.0327 - adj_r_squared: -0.5008
Epoch 42/1000
203/203 [==============================] - 0s 138us/step - loss: 58.9496 - mean_squared_error: 58.9496 - r_squared: -0.0112 - adj_r_squared: -0.4695
Epoch 43/1000
203/203 [==============================] - 0s 118us/step - loss: 58.9945 - mean_squared_error: 58.9945 - r_squared: -0.0197 - adj_r_squared: -0.4819
Epoch 44/1000
203/203 [==============================] - 0s 118us/step - loss: 59.1161 - mean_squared_error: 59.1161 - r_squared: -0.0326 - adj_r_squared: -0.5006
Epoch 45/1000
203/203 [==============================] - 0s 118us/step - loss: 58.9892 - mean_squared_error: 58.9892 - r_squared: -0.0245 - adj_r_squared: -0.4888
Epoch 46/1000
203/203 [==============================] - 0s 118us/step - loss: 59.0812 - mean_squared_error: 59.0812 - r_squared: -0.0235 - adj_r_squared: -0.4874
Epoch 47/1000
203/203 [=============

203/203 [==============================] - 0s 118us/step - loss: 59.1176 - mean_squared_error: 59.1176 - r_squared: -0.0385 - adj_r_squared: -0.5092
Epoch 92/1000
203/203 [==============================] - 0s 138us/step - loss: 59.2404 - mean_squared_error: 59.2404 - r_squared: -0.0321 - adj_r_squared: -0.4999
Epoch 93/1000
203/203 [==============================] - 0s 118us/step - loss: 58.9646 - mean_squared_error: 58.9646 - r_squared: -0.0115 - adj_r_squared: -0.4700
Epoch 94/1000
203/203 [==============================] - 0s 118us/step - loss: 59.5003 - mean_squared_error: 59.5003 - r_squared: -0.0649 - adj_r_squared: -0.5475
Epoch 95/1000
203/203 [==============================] - 0s 138us/step - loss: 59.0125 - mean_squared_error: 59.0125 - r_squared: -0.0179 - adj_r_squared: -0.4792
Epoch 96/1000
203/203 [==============================] - 0s 118us/step - loss: 59.1371 - mean_squared_error: 59.1371 - r_squared: -0.0358 - adj_r_squared: -0.5052
Epoch 97/1000
203/203 [=============

203/203 [==============================] - 0s 118us/step - loss: 58.9743 - mean_squared_error: 58.9743 - r_squared: -0.0309 - adj_r_squared: -0.4981
Epoch 142/1000
203/203 [==============================] - 0s 118us/step - loss: 59.0033 - mean_squared_error: 59.0033 - r_squared: -0.0143 - adj_r_squared: -0.4741
Epoch 143/1000
203/203 [==============================] - 0s 118us/step - loss: 59.0750 - mean_squared_error: 59.0750 - r_squared: -0.0228 - adj_r_squared: -0.4864
Epoch 144/1000
203/203 [==============================] - 0s 118us/step - loss: 59.2294 - mean_squared_error: 59.2294 - r_squared: -0.0395 - adj_r_squared: -0.5107
Epoch 145/1000
203/203 [==============================] - 0s 118us/step - loss: 59.7954 - mean_squared_error: 59.7954 - r_squared: -0.0998 - adj_r_squared: -0.5982
Epoch 146/1000
203/203 [==============================] - 0s 99us/step - loss: 58.8981 - mean_squared_error: 58.8981 - r_squared: -0.0076 - adj_r_squared: -0.4642
Epoch 147/1000
203/203 [========

203/203 [==============================] - 0s 118us/step - loss: 59.0879 - mean_squared_error: 59.0879 - r_squared: -0.0272 - adj_r_squared: -0.4928
Epoch 192/1000
203/203 [==============================] - 0s 118us/step - loss: 59.0061 - mean_squared_error: 59.0061 - r_squared: -0.0171 - adj_r_squared: -0.4781
Epoch 193/1000
203/203 [==============================] - 0s 118us/step - loss: 59.1423 - mean_squared_error: 59.1423 - r_squared: -0.0385 - adj_r_squared: -0.5091
Epoch 194/1000
203/203 [==============================] - 0s 118us/step - loss: 58.9752 - mean_squared_error: 58.9752 - r_squared: -0.0138 - adj_r_squared: -0.4732
Epoch 195/1000
203/203 [==============================] - 0s 99us/step - loss: 59.2501 - mean_squared_error: 59.2501 - r_squared: -0.0466 - adj_r_squared: -0.5209
Epoch 196/1000
203/203 [==============================] - 0s 99us/step - loss: 59.1094 - mean_squared_error: 59.1094 - r_squared: -0.0279 - adj_r_squared: -0.4937
Epoch 197/1000
203/203 [=========

...

203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 757/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 758/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 759/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 760/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 761/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 762/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan 

203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 812/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 813/1000
203/203 [==============================] - 0s 138us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 814/1000
203/203 [==============================] - 0s 99us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 815/1000
203/203 [==============================] - 0s 138us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 816/1000
203/203 [==============================] - 0s 99us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 817/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - 

203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 867/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 868/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 869/1000
203/203 [==============================] - 0s 99us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 870/1000
203/203 [==============================] - 0s 99us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 871/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 872/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - 

203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 922/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 923/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 924/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 925/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 926/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 927/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan 

203/203 [==============================] - 0s 138us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 977/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 978/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 979/1000
203/203 [==============================] - 0s 99us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 980/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 981/1000
203/203 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - r_squared: nan - adj_r_squared: nan
Epoch 982/1000
203/203 [==============================] - 0s 99us/step - loss: nan - mean_squared_error: nan - r_squared: nan - 

In [ ]:
def forward_sel(X,Y):
    m,n=X.shape
    for colno in range(0,n):
        X=X[:,0:colno+1]
        #run model from here..
    return None 